# Statistics with `statsmodels`

The Python package `statsmodels` uses `NumPy` and `SciPy` under the hood to do complex statistics in minimal lines of code. We will explore how to do ordinary least squares (OLS) fitting with `statsmodels` and diagnosing the quality of the fit in this notebook.

In [1]:
import numpy, pandas
import statsmodels as sms
import statsmodels.api as sms_api
import matplotlib.pyplot as pl
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Simplest case

Let's look at how OLS fitting works with an example of simulated data

Let's look at a model that we know can be defined as: $$ y = 1 + 0.5x + 10x^2 $$

In [7]:
#Define number of samples
nsamp = 100

#Define our independent data, i.e. X
x = numpy.linspace(0.1, 10, nsamp)
X = numpy.column_stack((x / x, x, x ** 2))

#Define our coefficients;
beta = numpy.array([1, 0.5, 10])

#Let's add some noise:
noise = numpy.random.normal(size = nsamp)

#Finally, given the above model, let's define our dependent variable, y
y = numpy.dot(X, beta) + noise

In [9]:
#Define an ordinary least squares (OLS) model object:
model = sms_api.OLS(y, X)

#Fit:
results = model.fit()

#Print summary of results:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.226e+06
Date:                Mon, 20 Jan 2020   Prob (F-statistic):          2.50e-240
Time:                        16:16:17   Log-Likelihood:                -144.24
No. Observations:                 100   AIC:                             294.5
Df Residuals:                      97   BIC:                             302.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2771      0.318      4.014      0.0

In [12]:
#We can extract the parameters like so:
print("The best-fit parameters are: {}".format(results.params))
#And errors:
print("Standard errors: {}".format(results.bse))

#And goodness-of-fit, which in this case is the R-squared metric:
print(r"$R^2$ is: {}".format(results.rsquared))

The best-fit parameters are: [  1.27713401   0.43581922  10.00261204]
Standard errors: [ 0.31818718  0.14542019  0.0139496 ]
$R^2$ is: 0.9999885247435667


The cool thing about `statsmodels` is that it also offers a way of defining symbolic formulae in its fitting, similar to the language `R`. This uses the `patsty` python package behind the scenes.

In [13]:
#For this to work, the data needs to be in a dataframe, with the column names representing the 
#name of the variable:
data_array = numpy.column_stack((y, X))
data = pandas.DataFrame(data_array, columns = ['y', 'const', 'x', 'x2'])

data.head()

,y,const,x,x2
0,0.875512,1.0,0.1,0.01
1,0.484899,1.0,0.2,0.04
2,3.328903,1.0,0.3,0.09
3,1.824917,1.0,0.4,0.16
4,5.292902,1.0,0.5,0.25


In [17]:
#We need a different module from the package first:
import statsmodels.formula.api as smf

#Let's use symbolic notation in our fitting this time:

model_2 = smf.ols(formula = 'y ~ const + x + x2 - 1', data = data)

results = model_2.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.226e+06
Date:                Mon, 20 Jan 2020   Prob (F-statistic):          2.50e-240
Time:                        16:38:09   Log-Likelihood:                -144.24
No. Observations:                 100   AIC:                             294.5
Df Residuals:                      97   BIC:                             302.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2771      0.318      4.014      0.0